<a href="https://colab.research.google.com/github/iyeedinho/Chatbot/blob/master/Creating_the_nlu_training_data(rasa_fr).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation

In [2]:
#install Rasa Open Source:
!pip install rasa
#Dependencies for spaCy
!pip install rasa[spacy]
! python -m spacy download en_core_web_sm
!python -m spacy download fr_core_news_sm
! python -m spacy link fr_core_news_sm fr
! python -m spacy link en_core_web_sm en 

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_sm')

✘ Link 'fr' already exists
To overwrite an existing link, use the --force flag


✘ Link 'en' already exists
To overwrite an existing link, use the --force flag



# Create a new project from rasa

In [3]:
!rasa init --no-prompt

Welcome to Rasa! 🤖

To get started quickly, an initial project will be created.
If you need some help, check out the documentation at https://rasa.com/docs/rasa.

Created project directory at '/content'.
Finished creating project structure.
Training an initial model...
Training Core model...
2020-06-13 18:12:50 INFO     root  - Generating grammar tables from /usr/lib/python3.6/lib2to3/Grammar.txt
2020-06-13 18:12:50 INFO     root  - Generating grammar tables from /usr/lib/python3.6/lib2to3/PatternGrammar.txt
Processed Story Blocks: 100% 5/5 [00:00<00:00, 3785.47it/s, # trackers=1]
Processed Story Blocks: 100% 5/5 [00:00<00:00, 1394.29it/s, # trackers=5]
Processed Story Blocks: 100% 5/5 [00:00<00:00, 363.58it/s, # trackers=20]
Processed Story Blocks: 100% 5/5 [00:00<00:00, 277.60it/s, # trackers=24]
Processed trackers: 100% 5/5 [00:00<00:00, 3114.27it/s, # actions=16]
Processed actions: 16it [00:00, 7927.80it/s, # examples=16]
Processed trackers: 100% 231/231 [00:01<00:00, 215.88it/s, #

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# View Your NLU Training Data





*   Displaying the data
*   NLU Training Data Management





In [0]:
!rm data/nlu.md

In [23]:
%%writefile data/nlu.md
#NLU_DATA
## intent:greet
- bonjour
- bonsoir
- salut
- allo
- coucou
- hey 

## intent:bye
- bonne nuit
- au revoir
- bonne journée
- bonne soirée
- à tout à l’heure
- à plus tard 
- à bientôt
- à demain
- adieu
- à la prochaine
## intent:thank
- Merci
- Merci beaucoup
- Merci bot
- Merci pour ça

## intent:affirm
- oui
- en effet
- bien sûr
- justement
- correcte
- exactement
- vrai

## intent:deny
- non
- jamais
- Je ne pense pas
- je n'aime pas ça
- pas du tout
- pas vraiment

## intent:bonne_humeur
- parfait
- très bon
- génial
- incroyable
- magnifique
- Je me sens très bien
- Je suis génial
- Je suis bien

## intent:mood_unhappy
- triste
- très triste
- malheureux
- mauvais
- très mauvais
- terrible
- affreux
- pas très bien
- extrêmement triste
- si triste
- pas bien

## intent:bot_challenge
- vous étes un robot?
- vous étes un être humain ?
- vous étes un humain ?
- vous étes un homme ?
- vous étes une femme ?
- je parle à un bot?
- je parle à un humain?
- tu es qui ?

Overwriting data/nlu.md


# Define Your Model Configuration

In [21]:
#!rm config.yml

%%writefile config.yml
# Configuration for Rasa NLU.
# https://rasa.com/docs/rasa/nlu/components/


language: "fr"
pipeline:
  - name: ConveRTTokenizer
  - name: ConveRTFeaturizer
  - name: RegexFeaturizer
  - name: LexicalSyntacticFeaturizer
  - name: CountVectorsFeaturizer
  - name: CountVectorsFeaturizer
    analyzer: "char_wb"
    min_ngram: 1
    max_ngram: 4
  - name: DIETClassifier
    epochs: 100
  - name: EntitySynonymMapper
  - name: ResponseSelector
    epochs: 100

Overwriting config.yml


In [0]:
# View Your Model Configuration
!cat config.yml

# Write Your First Stories

In [0]:
%%writefile /content/data/stories.md
## happy path
* greet
  - utter_greet
* mood_great
  - utter_happy

## sad path 1
* greet
  - utter_greet
* mood_unhappy
  - utter_cheer_up
  - utter_did_that_help
* affirm
  - utter_happy

## sad path 2
* greet
  - utter_greet
* mood_unhappy
  - utter_cheer_up
  - utter_did_that_help
* deny
  - utter_goodbye

## say goodbye
* goodbye
  - utter_goodbye
## say thanx
* thank
  - utter_thank

## bot challenge
* bot_challenge
  - utter_iamabot

In [0]:
#View Your Model 
!cat /content/data/stories.md

# Define a Domain

In [0]:
%%writefile domain.yml
intents:
  - greet
  - goodbye
  - thank
  - affirm
  - deny
  - mood_great
  - mood_unhappy
  - bot_challenge

responses:
  utter_greet:
  - text: "Bonjour! Comment ça va?"

  utter_cheer_up:
  - text: "Here is something to cheer you up:"
    image: "https://i.imgur.com/nGF1K8f.jpg"

  utter_did_that_help:
  - text: "Cela vous a-t-il aidé?"

  utter_happy:
  - text: "Super, continuez!"

  utter_goodbye:
  - text: "Au revoir"

  utter_iamabot:
  - text: "Je suis le ChatBot de support pour la solution LMS."

session_config:
  session_expiration_time: 60
  carry_over_slots_to_new_session: true

In [0]:
#View Your Domain 
!cat domain.yml